In [ ]:
import numpy as np
import csv
import os
import training_data_generation as tdg
import netcdf_read_write as nrw
import utils_intensity_map as uim

In [ ]:
def extract_power_from_polar(heat_source, theta, phi):
    theta_edges = (theta[1:] + theta[:-1]) / 2.0
    theta_edges = np.append(0.0, theta_edges)
    theta_edges = np.append(theta_edges, np.pi)
    phi_edges = (phi[1:] + phi[:-1]) / 2.0
    phi_edges = np.append(0.0, phi_edges)
    phi_edges = np.append(phi_edges, 2.0*np.pi)

    theta_grid, phi_grid = np.meshgrid(theta_edges, phi_edges)

    dphi = phi_grid[1:,1:] - phi_grid[:-1,1:]
    d_cos_theta = np.cos(theta_grid[1:,1:]) - np.cos(theta_grid[1:,:-1])
    domega = np.abs(dphi * d_cos_theta)

    total_pwr = np.sum(heat_source * domega)
    return total_pwr

In [ ]:
def define_dataset(num):
    dataset = {}
    dataset["counter"] = 0
    dataset["labels"] = num*[None]
    dataset["wavelength"] = np.zeros((num))
    dataset["power_emitted"] = np.zeros((num))
    dataset["power_reflected"] = np.zeros((num))
    dataset["power_absorbed"] = np.zeros((num))
    dataset["power_absorbed_no_plasma"] = np.zeros((num))
    dataset["power_cbet"] = np.zeros((num))
    dataset["power_cbet_error"] = np.zeros((num))
    dataset["intensity"] = np.zeros((num))
    dataset["intensity_no_plasma"] = np.zeros((num))
    dataset["ablation_pressure"] = np.zeros((num))
    dataset["has_cbet_data"] = num*[True]
    
    return dataset

In [ ]:
def extract_cbet_data(run_location, dataset, iconfig, rt):
    cbet_filename = "cbet_report.txt"
    
    line_is_data = False
    cbet_data = []
    with open(run_location + "/" + cbet_filename) as cbet_file:
        for line in cbet_file:
            if line_is_data:
                x = np.array(line.split())
                y = x.astype(float)
                cbet_data.append(y)
            if "***" in line:
                line_is_data = True

    cbet_data = np.stack(cbet_data[:-2], axis=0)
    
    power_emitted = cbet_data[:,1]
    dataset[rt]["power_emitted"][iconfig] = np.sum(power_emitted)
    power_reflected = cbet_data[:,2]
    dataset[rt]["power_reflected"][iconfig] = np.sum(power_reflected)
    power_cbet = cbet_data[:,3]
    dataset[rt]["power_cbet"][iconfig] = np.sum(power_cbet)
    power_absorbed = cbet_data[:,-3]
    tot_power_absorbed = np.sum(power_absorbed)
    dataset[rt]["power_cbet_error"][iconfig] = np.sum(power_emitted) - np.sum(power_reflected) - tot_power_absorbed
    
    return dataset

def extract_no_plasma_absorption(run_location, dataset, iconfig, rt, sys_params, facility_spec):
    surface_area = 4.0 * np.pi * (facility_spec['target_radius'] / 10000.0)**2

    parameters = nrw.read_general_netcdf(run_location + "/" + sys_params["ifriit_ouput_name"])
    intensity_map = parameters["intensity"]
    intensity_map_normalized, avg_flux = uim.imap_norm(intensity_map)
    dataset[rt]["intensity_no_plasma"][iconfig] = avg_flux
    dataset[rt]["power_absorbed_no_plasma"][iconfig] = avg_flux * surface_area
    return dataset

def extract_ablation_pressure(run_location, dataset, iconfig, rt, sys_params):

    file_name = sys_params["heat_source_nc"]
    hs_and_modes = nrw.read_general_netcdf(run_location + "/" + file_name)
    dataset[rt]["ablation_pressure"][iconfig] = hs_and_modes["average_flux"][0]
    
    dataset[rt]["power_absorbed"][iconfig] = extract_power_from_polar(hs_and_modes["heat_source"], hs_and_modes["theta"], hs_and_modes["phi"])
    
    return dataset

In [ ]:
def allocate_wavelengths(dataset, iconfig, rt):
    colour_tag = dataset[rt]["labels"][iconfig].split("_")[0]
    
    if colour_tag == "uv":
        dataset[rt]["wavelength"][iconfig] = 351.0
    elif colour_tag == "red":
        dataset[rt]["wavelength"][iconfig] = 1064.0
    elif colour_tag == "green":
        dataset[rt]["wavelength"][iconfig] = 520.0
    elif colour_tag == "arf":
        dataset[rt]["wavelength"][iconfig] = 192.0
    
    return dataset

In [ ]:
def allocate_data_labels(dir_config, run_types):
    split_name = dir_config.split("_")#[-1]
    run_type_ind = 0
    for ind_term in range(len(run_types)):
        word = run_types[ind_term]
        if word in split_name:
            run_type_ind = ind_term
    return run_type_ind

In [ ]:
diag_dir = "Data_240716a_pdd_param_scan"
ind_profile = 0
num_examples = 0
power_emitted = 160.86*10**12

dir_config_list = [name for name in os.listdir(diag_dir) if os.path.isdir(os.path.join(diag_dir, name))]
num_configs = len(dir_config_list)
rti_list = np.zeros(num_configs).astype(int)

run_types = ("normal", "nocbet", "bandwidth10", "bandwidth1", "bandwidth03", "bandwidth01")
run_type_labels = ("default", "no CBET", "bandwidth 10%", "bandwidth 1%", "bandwidth 0.3%", "bandwidth 0.1%")
num_run_types = len(run_types)
rt_counter = np.zeros(num_run_types).astype(int)

for iconfig in range(num_configs):
    dir_config = dir_config_list[iconfig]
    rti_list[iconfig] = allocate_data_labels(dir_config, run_types)
    rt_counter[rti_list[iconfig]]+=1
    
dataset = {}
for itype in range(num_run_types):
    dataset[run_types[itype]] = define_dataset(rt_counter[itype])
    print(rt_counter[itype])

sys_params = tdg.define_system_params(diag_dir)
dataset_params, facility_spec = tdg.define_dataset_params(int(num_examples))
surface_area = 4.0 * np.pi * (facility_spec['target_radius'] / 10000.0)**2


In [ ]:

for iconfig in range(num_configs):
    dir_config = dir_config_list[iconfig]
    rti = rti_list[iconfig]
    rt = run_types[rti]
    
    iconfig2 = dataset[rt]["counter"]
    dataset[rt]["labels"][iconfig2] = "_".join(dir_config.split("_")[1:])
    iex = dataset[rt]["labels"][iconfig2]
    config_location = sys_params["data_dir"] + "/" + sys_params["config_dir"] + str(iex)
    run_location = config_location + "/" + sys_params["sim_dir"] + str(ind_profile)
    
    dataset = allocate_wavelengths(dataset, iconfig2, rt)
    
    try:
        dataset = extract_cbet_data(run_location, dataset, iconfig2, rt)
    except:
        print("No CBET data found for: "+str(iex))
        dataset[rt]["has_cbet_data"][iconfig2]  = False

    dataset = extract_no_plasma_absorption(run_location, dataset, iconfig2, rt, sys_params, facility_spec)
    dataset = extract_ablation_pressure(run_location, dataset, iconfig2, rt, sys_params)
    dataset[rt]["counter"]+=1


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.ion();

plot_file_type = ".png"

trends_aspect_ratio = [6, 4]
trends_axis_limits = [0.3, 0.15, 0.65, 0.8]

marker_size = 8
plot_markers = ("o", "X", "*","*","*","*")

print(dataset[run_types[0]].keys())

In [ ]:
fig1 = plt.figure(num=1, figsize=trends_aspect_ratio, facecolor='white')
ax1 = fig1.add_axes(trends_axis_limits)
fig2 = plt.figure(num=2, figsize=trends_aspect_ratio, facecolor='white')
ax2 = fig2.add_axes(trends_axis_limits)
fig3 = plt.figure(num=3, figsize=trends_aspect_ratio, facecolor='white')
ax3 = fig3.add_axes(trends_axis_limits)
fig4 = plt.figure(num=4, figsize=trends_aspect_ratio, facecolor='white')
ax4 = fig4.add_axes(trends_axis_limits)
#fig5 = plt.figure(num=5, figsize=trends_aspect_ratio, facecolor='white')
#ax5 = fig5.add_axes(trends_axis_limits)

for itype in range(num_run_types):
    rt = run_types[itype]
    ax1.plot(dataset[rt]["wavelength"], dataset[rt]["ablation_pressure"], plot_markers[itype], ms=marker_size)
    ax2.plot(dataset[rt]["wavelength"], dataset[rt]["power_absorbed"] / power_emitted * 100.0, plot_markers[itype], ms=marker_size)
    ax3.plot(dataset[rt]["wavelength"], dataset[rt]["power_cbet"]*10**12 / power_emitted * 100.0, plot_markers[itype], ms=marker_size)
    ax4.plot(dataset[rt]["wavelength"], dataset[rt]["power_cbet_error"]*10**12 / power_emitted * 100.0, plot_markers[itype], ms=marker_size)
    #print(rt, dataset[rt]["labels"])
    #print(rt, dataset[rt]["power_absorbed_no_plasma"])

ax1.set_ylim(bottom=0)
ax1.set_xlabel("Wavelength")
ax1.set_ylabel("Ablation Pressure (Mbar)")
ax1.legend(run_type_labels)
fig1.savefig(sys_params["figure_location"]+"/trends_ablation_pressure" + plot_file_type, dpi=300, bbox_inches="tight")

ax2.set_ylim([0, 100])
ax2.set_xlabel("Wavelength")
ax2.set_ylabel("Power Absorbed (%)")
ax2.legend(run_type_labels)
fig2.savefig(sys_params["figure_location"]+"/trends_power_absorbed" + plot_file_type, dpi=300, bbox_inches="tight")

ax3.set_ylim([0, 120])
ax3.set_xlabel("Wavelength")
ax3.set_ylabel("Emitted Power Transferred via CBET (%)")
ax3.legend(run_type_labels)
fig3.savefig(sys_params["figure_location"]+"/trends_cbet_transfer" + plot_file_type, dpi=300, bbox_inches="tight")

#ax4.set_ylim([0, 120])
ax4.set_xlabel("Wavelength")
ax4.set_ylabel("CBET error (%)")
ax4.legend(run_type_labels)
fig4.savefig(sys_params["figure_location"]+"/trends_cbet_error" + plot_file_type, dpi=300, bbox_inches="tight")

In [ ]:
fig1 = plt.figure(num=1, figsize=trends_aspect_ratio, facecolor='white')
ax1 = fig1.add_axes(trends_axis_limits)
fig2 = plt.figure(num=2, figsize=trends_aspect_ratio, facecolor='white')
ax2 = fig2.add_axes(trends_axis_limits)
fig3 = plt.figure(num=3, figsize=trends_aspect_ratio, facecolor='white')
ax3 = fig3.add_axes(trends_axis_limits)
fig4 = plt.figure(num=4, figsize=trends_aspect_ratio, facecolor='white')
ax4 = fig4.add_axes(trends_axis_limits)

itype = 4
rt = run_types[itype]
counter = 0
for iconf in range(len(dataset[rt]["wavelength"])):
    if dataset[rt]["wavelength"][iconf] == 351:
        ax1.plot(dataset[rt]["wavelength"][iconf], dataset[rt]["ablation_pressure"][iconf], plot_markers[counter], ms=marker_size, label=dataset[rt]["labels"][iconf]) #, fillstyle="none"
        ax2.plot(dataset[rt]["wavelength"][iconf], dataset[rt]["power_absorbed"][iconf] / power_emitted * 100.0, plot_markers[counter], ms=marker_size, label=dataset[rt]["labels"][iconf])
        ax3.plot(dataset[rt]["wavelength"][iconf], dataset[rt]["power_cbet"][iconf]*10**12 / power_emitted * 100.0, plot_markers[counter], ms=marker_size, label=dataset[rt]["labels"][iconf])
        ax4.plot(dataset[rt]["wavelength"][iconf], dataset[rt]["power_cbet_error"][iconf]*10**12 / power_emitted * 100.0, plot_markers[counter], ms=marker_size, label=dataset[rt]["labels"][iconf])
        counter+=1

ax1.set_xlim(right=500)
ax1.set_xlabel("Wavelength")
ax1.set_ylabel("Ablation Pressure (Mbar)")
ax1.legend(loc="best")
fig1.savefig(sys_params["figure_location"]+"/bandwidth_convergence_test_ablation_pressure" + plot_file_type, dpi=300, bbox_inches="tight")
ax2.set_xlim(right=500)
ax2.set_xlabel("Wavelength")
ax2.set_ylabel("Power Absorbed (%)")
ax2.legend()
fig2.savefig(sys_params["figure_location"]+"/bandwidth_convergence_test_power_absorbed" + plot_file_type, dpi=300, bbox_inches="tight")
ax3.set_xlim(right=500)
ax3.set_xlabel("Wavelength")
ax3.set_ylabel("Emitted Power Transferred via CBET (%)")
ax3.legend()
fig3.savefig(sys_params["figure_location"]+"/bandwidth_convergence_test_CBET_transfer" + plot_file_type, dpi=300, bbox_inches="tight")
ax4.set_xlim(right=500)
ax4.set_xlabel("Wavelength")
ax4.set_ylabel("CBET error (%)")
ax4.legend()
fig4.savefig(sys_params["figure_location"]+"/bandwidth_convergence_test_CBET_error" + plot_file_type, dpi=300, bbox_inches="tight")

# Compute time estimate

In [ ]:

# duration of simulation
t_sim = 0.40
# number of OMP threads
num_parallel_OMP = 10
# number of MPI threads is equal to number of beams
num_ports = 48
beams_per_port = 4

# CPU time required
cpu_hours = t_sim * num_parallel_OMP * num_ports * beams_per_port
print(cpu_hours)

In [ ]:
print(run_types)
print(run_type_labels)
rt = run_types[0]
print(rt, dataset[rt]["labels"])

In [ ]:

labels = ['red', 'green', 'uv', 'ArF']
cputime_default = np.array([0, 419, 302, 257]) / 60
cputime_nocbet = np.array([144, 111, 96, 90]) / 60
cputime_bandwidth = np.array([1253, 1578, 1308, 1193]) / 60
cputime_splitbeams = np.array([1054]) / 60
cputime_splitbeams_nocbet = np.array([110]) / 60

labels = ['default', 'No CBET', 'fuse quads (FQ)', 'FQ, no CBET', "FQ, bandwidth"]
cputime_types = np.array([1054, 110, 302, 96, 1308]) / 60 / 60 * num_parallel_OMP * num_ports
cputime_types[0:2] = cputime_types[0:2] * beams_per_port

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x, cputime_types, width, label='default')
#rects2 = ax.bar(x + width/2, cputime_nocbet, width, label='no CBET')
#rects3 = ax.bar(x + width/2, cputime_bandwidth, width, label='bandwidth')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('CPU hours')
ax.set_xlabel('')
ax.set_xticks(x)
ax.set_xticklabels(labels)
#ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(int(height)),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
#autolabel(rects2)

fig.tight_layout()

plt.show()